In [1]:
from ontobio.ontol_factory import OntologyFactory


In [4]:
ofa = OntologyFactory()

In [19]:
ont = ofa.create('merged.json')


In [6]:
#ecto = ofa.create('ecto.json')

In [20]:
HAS_EXPOSURE_STIMULUS = 'RO:0002309'
HAS_ROLE = 'RO:0000087'

In [9]:
#exo = ofa.create('exposure-all-ecto.json')

In [53]:
tuples = []

isa = ont.subontology(relations=['subClassOf'])
all_symptoms = set()

for ex in ont.nodes():
    exl = ont.label(ex)
    
    # hacky way to get rid of additional route-specific classes
    if exl is None:
        continue
    symptoms = ont.parents(ex, ['RO:0002304'])
    all_symptoms.update(symptoms)
    for ph in symptoms:
        tuples.append( (ex,exl,ph) )
    roles = set()
    for ld in ont.logical_definitions(ex):
        for _,chem in ld.restrictions:
            tuples.append( (ex,exl,chem) )
            #print('CHEM= {}'.format(chem))
    
    
tuples[0:10]

[('UBERON:0005177', 'trunk region element', 'UBERON:0002100'),
 ('http://purl.obolibrary.org/obo/ECTO_0002261',
  'tetrahydrofuran exposure',
  'CHEBI:26911'),
 ('http://purl.obolibrary.org/obo/ECTO_0001482',
  'exposure to iooxitalamic acid',
  'CHEBI:83517'),
 ('http://purl.obolibrary.org/obo/ECTO_0001848',
  'exposure to p-nitrophenyl group',
  'CHEBI:61595'),
 ('http://purl.obolibrary.org/obo/ECTO_0001276',
  'exposure to (trifluoromethyl)benzene',
  'CHEBI:36810'),
 ('http://purl.obolibrary.org/obo/ECTO_0000225',
  'exposure to lapatinib',
  'CHEBI:49603'),
 ('http://purl.obolibrary.org/obo/ECTO_0001713',
  'Dicyclopentadiene exposure',
  'CHEBI:34695'),
 ('http://purl.obolibrary.org/obo/ECTO_0001150',
  'prenatal exposure to chemical entity',
  'UBERON:0000068'),
 ('http://purl.obolibrary.org/obo/ECTO_0001150',
  'prenatal exposure to chemical entity',
  'http://purl.obolibrary.org/obo/ECTO_0000231'),
 ('http://purl.obolibrary.org/obo/ECTO_0000322',
  'exposure to quetiapine',
  

In [54]:
from ontobio.assoc_factory import AssociationSetFactory
afa = AssociationSetFactory()


In [55]:
aset = afa.create_from_tuples(tuples, ontology=ont)

In [56]:
aset.annotations('http://purl.obolibrary.org/obo/ECTO_0001731')

['http://purl.obolibrary.org/obo/Wikidata_Q568321',
 'http://purl.obolibrary.org/obo/Wikidata_Q5445',
 'http://purl.obolibrary.org/obo/Wikidata_Q21120220',
 'http://purl.obolibrary.org/obo/Wikidata_Q21120214',
 'http://purl.obolibrary.org/obo/Wikidata_Q192120',
 'http://purl.obolibrary.org/obo/Wikidata_Q2266426',
 'http://purl.obolibrary.org/obo/Wikidata_Q133244',
 'http://purl.obolibrary.org/obo/Wikidata_Q474187',
 'http://purl.obolibrary.org/obo/Wikidata_Q6402731',
 'http://purl.obolibrary.org/obo/Wikidata_Q1349821',
 'http://purl.obolibrary.org/obo/Wikidata_Q950591',
 'CHEBI:51398']

In [57]:
[role] = ont.search('role')
role

'CHEBI:50906'

In [59]:
def etest(r):
    sample = aset.query([r])
    results = aset.enrichment_test(sample, threshold=0.005, labels=True, hypotheses=all_symptoms)
    for result in results:
        print('{} {} -> {} {} {}'.format(r, ont.label(r), result['c'], result['n'], result['p']))
        
for r in isa.descendants(role):
    etest(r)

CHEBI:24852 insecticide -> http://purl.obolibrary.org/obo/Wikidata_Q40878 diarrhea 0.00015953418022636944
CHEBI:24852 insecticide -> http://purl.obolibrary.org/obo/Wikidata_Q4424146 salivation 0.001854021535352299
CHEBI:24852 insecticide -> http://purl.obolibrary.org/obo/Wikidata_Q3002092 abdominal cramps 0.001854021535352299
CHEBI:24852 insecticide -> http://purl.obolibrary.org/obo/Wikidata_Q596899 miosis 0.003148051693509966
CHEBI:76773 EC 3.1.1.* (carboxylic ester hydrolase) inhibitor -> http://purl.obolibrary.org/obo/Wikidata_Q4424146 salivation 8.03162483774844e-05
CHEBI:76773 EC 3.1.1.* (carboxylic ester hydrolase) inhibitor -> http://purl.obolibrary.org/obo/Wikidata_Q3002092 abdominal cramps 8.03162483774844e-05
CHEBI:76773 EC 3.1.1.* (carboxylic ester hydrolase) inhibitor -> http://purl.obolibrary.org/obo/Wikidata_Q596899 miosis 0.00023369747758478703
CHEBI:76773 EC 3.1.1.* (carboxylic ester hydrolase) inhibitor -> http://purl.obolibrary.org/obo/Wikidata_Q21109840 chest tightne